In [1]:
import numpy as np

def gini_impurity(y):
    classes, counts = np.unique(y, return_counts=True)
    prob_sq = (counts / len(y)) ** 2
    return 1 - np.sum(prob_sq)

In [2]:
def generate_thresholds(feature_col):
    unique_vals = np.unique(feature_col)
    thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2
    return thresholds

In [3]:
def split_data(X, y, feature_idx, threshold):
    left_mask = X[:, feature_idx] <= threshold
    right_mask = X[:, feature_idx] > threshold
    return (X[left_mask], y[left_mask]), (X[right_mask], y[right_mask])

In [4]:
def decision_tree_classifier(X, y, depth=0, max_depth=5, min_samples=2):
    # Base cases
    if len(y) < min_samples or depth >= max_depth or len(np.unique(y)) == 1:
        return {'leaf': True, 'class': np.bincount(y).argmax()}

    m, n = X.shape
    best_feature = None
    best_threshold = None
    best_impurity = float('inf')
    best_split = None

    for feature_idx in range(n):
        thresholds = generate_thresholds(X[:, feature_idx])
        for threshold in thresholds:
            (X_left, y_left), (X_right, y_right) = split_data(X, y, feature_idx, threshold)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            impurity = (len(y_left) / m) * gini_impurity(y_left) + (len(y_right) / m) * gini_impurity(y_right)

            if impurity < best_impurity:
                best_impurity = impurity
                best_feature = feature_idx
                best_threshold = threshold
                best_split = ((X_left, y_left), (X_right, y_right))

    if best_split is None:
        return {'leaf': True, 'class': np.bincount(y).argmax()}

    left_tree = decision_tree_classifier(*best_split[0], depth + 1, max_depth, min_samples)
    right_tree = decision_tree_classifier(*best_split[1], depth + 1, max_depth, min_samples)

    return {
        'leaf': False,
        'feature': best_feature,
        'threshold': best_threshold,
        'left': left_tree,
        'right': right_tree
    }

In [5]:
def predict_single(x, tree):
    while not tree['leaf']:
        if x[tree['feature']] <= tree['threshold']:
            tree = tree['left']
        else:
            tree = tree['right']
    return tree['class']

def predict(X, tree):
    return np.array([predict_single(x, tree) for x in X])

In [6]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([0, 1, 1, 0])  

tree = decision_tree_classifier(X, y, max_depth=2)

# Make predictions
preds = predict(X, tree)
print("Predictions:", preds)

Predictions: [0 1 1 0]


In [7]:
## decision tree regressor

In [8]:
def mse_impurity(y):
    y_mean = np.mean(y)
    return np.sum((y - y_mean) ** 2)

In [9]:
def generate_thresholds_regressor(feature_column):
    unique_vals = np.unique(feature_column)
    thresholds = [(unique_vals[i] + unique_vals[i + 1]) / 2 for i in range(len(unique_vals) - 1)]
    return thresholds

In [10]:
def decision_tree_classifier(X, y, depth=0, max_depth=5, min_samples=2):
    # Base cases
    if len(y) < min_samples or depth >= max_depth:
        return {'leaf': True, 'class': np.mean(y)}

    m, n = X.shape
    best_feature = None
    best_threshold = None
    best_impurity = float('inf')
    best_split = None

    for feature_idx in range(n):
        thresholds = generate_thresholds_regressor(X[:, feature_idx])
        for threshold in thresholds:
            (X_left, y_left), (X_right, y_right) = split_data(X, y, feature_idx, threshold)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            impurity = (len(y_left) / m) * mse_impurity(y_left) + (len(y_right) / m) * mse_impurity(y_right)

            if impurity < best_impurity:
                best_impurity = impurity
                best_feature = feature_idx
                best_threshold = threshold
                best_split = ((X_left, y_left), (X_right, y_right))

    if best_split is None:
        return {'leaf': True, 'class': np.mean(y)}

    left_tree = decision_tree_classifier(*best_split[0], depth + 1, max_depth, min_samples)
    right_tree = decision_tree_classifier(*best_split[1], depth + 1, max_depth, min_samples)

    return {
        'leaf': False,
        'feature': best_feature,
        'threshold': best_threshold,
        'left': left_tree,
        'right': right_tree
    }

In [11]:
def predict_single(x, tree):
    while not tree['leaf']:
        if x[tree['feature']] <= tree['threshold']:
            tree = tree['left']
        else:
            tree = tree['right']
    return tree['class']

def predict(X, tree):
    return np.array([predict_single(x, tree) for x in X])

In [12]:
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([1.2, 1.9, 3.0, 3.9, 5.1])

tree = decision_tree_classifier(X, y, max_depth=2)

preds = predict(X, tree)
print("Predictions:", preds)

Predictions: [1.55 1.55 3.   3.9  5.1 ]
